In [34]:
import openai
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
import os
import ipywidgets as widgets
import time
from IPython.display import display, clear_output

# %run "./utils/datehelper.ipynb" import DateHelpers


In [35]:
load_dotenv()
client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))

In [36]:
class Infantil:
    
    def __init__(self):
        self.date = DateHelpers.input_date()
        self.date_str = DateHelpers.data_por_extenso(self.date.split('/')[0], self.date.split('/')[1], self.date.split('/')[2])
        self.theme = self.get_theme()
        
        self.news1_header = self.get_news1_header()
        self.news1_text = self.get_news1_text()
        self.news1_image = ''
        
        self.news2_text = self.get_curiosity_text()
        self.news2_image = self.get_curiosity_image(self, self.news2_text)
        self.news2_header = 'Você sabia?'
        
        self.news3_text = self.get_curiosity_text()
        self.news3_header = 'Você sabia?'
        self.news3_image = self.get_curiosity_image(self, self.news3_text)


    
    def get_theme(self):
        url = 'https://www.ponteiro.com.br/todas_datas.php'
        headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome / 86.0.4240.198Safari / 537.36"}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        all_dates = soup.find_all('tr')
        
        calendar = []
        for date in all_dates:
            day = date.find('td', width=" 40").get_text().strip() if date.find('td', width=" 40") != None else None
            val = date.find('td', width="490").get_text().strip() if date.find('td', width="490") != None else None
            calendar.append({"dia": self.date, "comemora": 'Personalizar tema'})
            if day == self.date[0:6] and val != None:
                calendar.append({"dia": day, "comemora": val})
        
        counter = 0
        for c in calendar:
            print(f'{counter} >> {c['comemora']}')
        
        time.sleep(1)
        
        themeNumber = int(input('Digite o número do tema (##)'))
        if themeNumber == 1:
            return input(f'Digite o tema do dia {self.date}')
        else:
            return calendar[themeNumber-1]['comemora']



    def get_news1_header(self):
        parts = self.date_str.split(' ')
        news1_header = f'{parts[1]} de {parts[3]}: {self.theme}'
        return news1_header



    def get_news1_text(self):
        content = f'Escreva um texto voltado para crianças recém alfabetizadas com o tema {self.theme}. ' 
        content += f'Esse texto estará na seção infantil do dia {self.date[0:6]} ao lado de uma imagem em preto em branco sobre o tema (que será colorida).'
        content += f'O texto deve conter entre 1000 e 1200 caracteres. Escreva estritamente o corpo do texto.'
        
        client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))
        message = [{
            'role': 'user',
            'content': content
        }]
        response = client.chat.completions.create(
            messages=message, 
            model="gpt-4o-mini", 
            max_tokens=10000, 
            temperature=0, 
        )
        news_text = response.choices[0].message.content
        return news_text



    def get_curiosity_text(self):
        theme = input('Dgite o tema da curiosidade:')
        content = f'Aponte-me uma curiosidade para inserir na seção infantil de um jornal. '
        content += f'Forneça, como resposta, estritamente a curiosidade da maneira como ela deve ser escrita em um texto contento entre 1000 e 1200 caracteres. '
        content += f'O tema é {theme}.'
        client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))
        message = [{
            'role': 'user',
            'content': content
        }]
        response = client.chat.completions.create(
            messages=message, 
            model="gpt-4o-mini", 
            max_tokens=10000, 
            temperature=1, 
        )
        curiosity_text = response.choices[0].message.content
        return curiosity_text



    def get_curiosity_image(self, curiosity):
        prompt = f'Crie uma imagem infantil para ser colocara junto ao texto a seguir na seção infantil de um jornal de grande circulação.\n\n' 
        prompt += f'{curiosity}'
        
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            quality="standard",
            n=1,
        )
        image_url = response.data[0].url
        return image_url

